<a href="https://colab.research.google.com/github/FaisalT0435/Artefak-TA/blob/main/Testing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Feb  9 09:18:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/Mask_RCNN/Dataset')

In [ ]:
import torch

In [ ]:

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 1e-5
WEIGHT_DECAY = 5e-4
BATCH_SIZE = 8
NUM_EPOCHS = 1
NUM_WORKERS = 4
CHECKPOINT_FILE = "1152KaggleBest.pth.tar"
PIN_MEMORY = True
SAVE_MODEL = False
LOAD_MODEL = False
TRAIN_DIR = 'train'
VALID_DIR = 'valid'
TEST_DIR = 'test'
IMAGE_SIZE = [416,416]

In [ ]:
import torch
from tqdm import tqdm
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import datasets, models

In [ ]:
def get_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(
            in_features, num_classes=5)
    model.to(DEVICE)
    
    return model

In [ ]:
model = get_model()
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, nesterov=True, weight_decay=1e-4)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

In [ ]:
def load_checkpoint(checkpoint, model, optimizer, lr):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    #optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

In [ ]:
model = get_model()
  
load_checkpoint(torch.load('/content/drive/MyDrive/Dataset/1152KaggleBest_second_5.pth.tar'), model, optimizer, LEARNING_RATE)

model.eval()

=> Loading checkpoint


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [ ]:
import cv2

In [ ]:
def predict_single_frame(frame):
    scale_percent = 60 # percent of original size
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)
      
    # resize image
    images = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)


    images = cv2.resize(frame, IMAGE_SIZE, cv2.INTER_LINEAR)
    images = torch.as_tensor(images, dtype=torch.float32).unsqueeze(0)
    images = images.swapaxes(1, 3).swapaxes(2, 3)
    images = list(image.to(DEVICE) for image in images)
    
    with torch.no_grad():
      pred = model(images)
    
    # print(pred)
    
    im = images[0].swapaxes(0, 2).swapaxes(0, 1).detach().cpu().numpy().astype(np.float32)
    im2 = np.zeros_like(im).astype(np.float32)
    x = "Not Pest"
    for i in range(len(pred[0]['masks'])):
        label = []
        msk=pred[0]['masks'][i,0].detach().cpu().numpy()
        scr=pred[0]['scores'][i].detach().cpu().numpy()
        box=pred[0]['boxes'][i].detach().cpu().numpy()
        lbl=pred[0]['labels'][i].detach().cpu().numpy()
        
        if scr>0.8 :
            if (lbl == 1) or (lbl == 3) or (lbl == 4) :
              x = "Pest"
              
            else :
              x = "Not Pest"
            
            cv2.rectangle(im, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0,0,1), 2)
            cv2.putText(im, "{0:.2f}%".format(scr*100), (int(box[0]+10), int(box[1])+40), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0,0,1), 2, cv2.LINE_AA)
            cv2.putText(im, x , (int(box[0]+4), int(box[1])+15), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0,0,1), 2, cv2.LINE_AA)
            im2[:,:,0][msk>0.87] = np.random.uniform(0,1)
            im2[:, :, 1][msk > 0.87] = np.random.uniform(0,1)
            im2[:, :, 2][msk > 0.87] = np.random.uniform(0,1)


      
   
       
    return (cv2.addWeighted(im, 0.8, im2, 0.2,0)*255).astype(np.uint8),x

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
!pip install pycocotools
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import cv2
import numpy as np
from pycocotools.coco import COCO
from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
import pandas as pd

In [ ]:
hasil_pre = []
folder_dir = "/content/drive/MyDrive/Dataset/Uji/"
        print(os.listdir(folder_dir))
        for images in os.listdir(folder_dir):
        
            # check if the image ends with png
            # if (images.endswith(".png")):
                # print(images)
              cap = cv2.VideoCapture(folder_dir+images)
              model.train(False)

              if (cap.isOpened()== False): 
                  print("Error opening video stream or file")

              images = []   
              while(cap.isOpened()):
                  ret, frame = cap.read()
                  if ret == True:
                      result_frame, predict = predict_single_frame(frame)
                      images.append(result_frame)
                      hasil_pre.append (predict)
                      # cv2_imshow(result_frame)
                  else: 
                      break

              cap.release()

In [ ]:
labels = pd.read_csv('/content/Data Benar.csv', sep=',')


In [ ]:
def get_label(row):
  for c in labels.columns:
    if row[c]==1:
      return c

In [ ]:
test_labels = labels.apply(get_label, axis=1)

In [ ]:
df_hasil_pred = pd.DataFrame(hasil_pre)
df_hasil_pred [1] = pd.DataFrame(test_labels)

df_hasil_pred

In [ ]:
# Faster RCNN

actual = df_hasil_pred[0]
predicted = df_hasil_pred[1]
from sklearn.metrics import classification_report
print(classification_report(actual, predicted, target_names=['Belalang', 'Sehat', 'Ulat', 'Wereng']))

              precision    recall  f1-score   support

    Belalang       1.00      1.00      1.00         3
       Sehat       1.00      0.80      0.89         5
        Ulat       1.00      1.00      1.00        10
      Wereng       0.92      1.00      0.96        11

    accuracy                           0.97        29
   macro avg       0.98      0.95      0.96        29
weighted avg       0.97      0.97      0.96        29



In [ ]:
# Fast RCNN

actual = df_hasil_pred[0]
predicted = df_hasil_pred[1]
from sklearn.metrics import classification_report
print(classification_report(actual, predicted, target_names=['Belalang', 'Sehat', 'Ulat', 'Wereng']))

              precision    recall  f1-score   support

    Belalang       1.00      1.00      1.00         3
       Sehat       1.00      0.67      0.80         6
        Ulat       1.00      1.00      1.00        10
      Wereng       0.83      1.00      0.91        10

    accuracy                           0.93        29
   macro avg       0.96      0.92      0.93        29
weighted avg       0.94      0.93      0.93        29



In [ ]:
# Mask RCNN

actual = df_hasil_pred[0]
predicted = df_hasil_pred[1]
from sklearn.metrics import classification_report
print(classification_report(actual, predicted, target_names=['Belalang', 'Sehat', 'Ulat', 'Wereng']))

              precision    recall  f1-score   support

    Belalang       1.00      1.00      1.00         3
       Sehat       1.00      0.50      0.67         8
        Ulat       0.90      1.00      0.95         9
      Wereng       0.75      1.00      0.86         9

    accuracy                           0.86        29
   macro avg       0.91      0.88      0.87        29
weighted avg       0.89      0.86      0.85        29



In [ ]:
!pip install paho-mqtt
!pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 KB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62132 sha256=5ec1c177803e2dcae5244993e9f7fcf9a6138ab503bd66913df70330f11c6d59
  Stored in directory: /root/.cache/pip/wheels/6a/48/01/c895c027e9b9367ec5470fbf371ee56e795a49ac6a19aa4c9f
Successfully built paho-mqtt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import random
import time

from paho.mqtt import client as mqtt_client

broker = 'broker.emqx.io'
port = 1883
topic1 = "detect/Pest"
# topic0 = "detect/healthy"
# generate client ID with pub prefix randomly
client_id = f'python-mqtt-{random.randint(0, 1000)}'
username = 'emqx'
password = 'public'

def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(client_id)
    client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client


def publish(client):
    
    msg_count = 0
    msg = ''
    hasil_pre = []
 
  # get the path/directory
    




    while True:
        time.sleep(1)
        # pred = predict()
        # print (pred)

        folder_dir = "/content/drive/MyDrive/Mask_RCNN/Dataset/Uji/"
        print(os.listdir(folder_dir))
        for images in os.listdir(folder_dir):
        
            # check if the image ends with png
            # if (images.endswith(".png")):
                # print(images)
              cap = cv2.VideoCapture(folder_dir+images)
              model.train(False)

              if (cap.isOpened()== False): 
                  print("Error opening video stream or file")

              images = []   
              while(cap.isOpened()):
                  ret, frame = cap.read()
                  if ret == True:
                      result_frame, predict = predict_single_frame(frame)
                      images.append(result_frame)
                      hasil_pre.append (predict)
                      # cv2_imshow(result_frame)
                  else: 
                      break

              cap.release()
        deteksi = []
        for i in hasil_pre:
            if (i == 'Pest')  :
                j = 'Pest'
                deteksi.append(j)
            else:
                j = 'healthy'
                deteksi.append(j)
        # print (deteksi)
    
        for x in deteksi:
            if (x == 'Pest'):
                msg = "Pest"
                result = client.publish(topic1, msg)
                status = result[0]
                if status == 0:
                  print(f"Send `{msg}` to topic `{topic1}`")
            else:
                  print(f"Failed to send message to topic {topic1}")


def run():
    client = connect_mqtt()
    client.loop_start()
    publish(client)


In [ ]:
run()

Streaming output truncated to the last 5000 lines.
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Failed to send message to topic detect/Pest
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/Pest`
Send `Pest` to topic `detect/P

KeyboardInterrupt: ignored